## Загрузка данных

In [1]:
from sklearn.datasets import fetch_20newsgroups


c = 0.15  # отношение количества аномальных экземпляров к нормальным

# загужаем данные
normal_data = fetch_20newsgroups(subset='all', categories=['sci.electronics'],
                               shuffle=True, random_state=123, 
                               remove=['headers', 'footers'], return_X_y=True)[0]
anomal_data = fetch_20newsgroups(subset='all', categories=['talk.politics.mideast'],
                               shuffle=True, random_state=123,
                               remove=['headers', 'footers'],
                               return_X_y=True)[0][:int(c * len(normal_data)) + 1]

# # приводим к одинаковой длине
# min_len = max(len(normal_data), len(anomal_data))
# normal_data = normal_data[:min_len]
# anomal_data = anomal_data[:min_len]
print("Количество нормальных экземпляров = {}".format(len(normal_data)))
print("Количество аномальных экземпляров = {}".format(len(anomal_data)))

Количество нормальных экземпляров = 984
Количество аномальных экземпляров = 148


## Предобработка текстовых данных

In [2]:
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.utils import tokenize
import nltk; nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


def strip_html_tags(text):
    """Удаление html tags из текста."""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def preprocess_text(text):
    text = strip_html_tags(text)  # удаление html tags
    text = strip_non_alphanum(text) # заменили все небуквенные символы на пробел
    text = strip_numeric(text) # удалили все цифры
    text = remove_stopwords(text) # удалили все стоп-слова
    text = strip_short(text, minsize=2) # удалили короткие слова
    word_list = list(tokenize(text, deacc=True, to_lower=True)) # токенизация, deacc - избавляет от ударений
    word_list = [WordNetLemmatizer().lemmatize(word) for word in word_list] # лемматизация
    return ' '.join(word for word in word_list)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
normal_data = [preprocess_text(text) for text in normal_data]
anomal_data = [preprocess_text(text) for text in anomal_data]
all_data = normal_data + anomal_data

In [4]:
import numpy as np


len_all_data = np.array([len(text.split(' ')) for text in all_data])
# print(len(len_all_data[len_all_data <= 150]))
print("mean length of sentence: " + str(len_all_data.mean()))
print("max length of sentence: " + str(len_all_data.max()))
print("std dev length of sentence: " + str(len_all_data.std()))

mean length of sentence: 110.97968197879858
max length of sentence: 6252
std dev length of sentence: 270.34016089125856


## Tfidf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(all_data)

normal_sequences = vectorizer.transform(normal_data).toarray()
anomal_sequences = vectorizer.transform(anomal_data).toarray()
print(normal_sequences.shape)
print(anomal_sequences.shape)

(984, 14685)
(148, 14685)


## Формирование выборок

In [6]:
from sklearn.utils import shuffle


x = np.concatenate((normal_sequences, anomal_sequences))
y = np.array([False] * normal_sequences.shape[0] + [True] * anomal_sequences.shape[0])

all_data, x, y = shuffle(all_data, x, y, random_state=123)
print("Всего экземпляров = {}".format(len(all_data)))
print("(Кол-во текстов, число признаков текста) = {}".format(x.shape))
print("Кол-во меток = {}".format(len(y)))
print("Кол-во нормальных экземпляров = {}".format(normal_sequences.shape[0]))
print("Кол-во аномальных экземпляров = {}".format(anomal_sequences.shape[0]))


Всего экземпляров = 1132
(Кол-во текстов, число признаков текста) = (1132, 14685)
Кол-во меток = 1132
Кол-во нормальных экземпляров = 984
Кол-во аномальных экземпляров = 148


## Функция Cosine similarity

In [7]:
def cosine_similarity(x_predict, x):
    if type(x_predict) is np.ndarray:
        flat_output = np.reshape(x_predict, (np.shape(x)[0], -1))
        flat_input = np.reshape(x_predict, (np.shape(x)[0], -1))
        sum = np.sum(flat_output * flat_input, -1)
        norm1 = np.linalg.norm(flat_output, axis=-1) + 0.000001
        norm2 = np.linalg.norm(flat_input, axis=-1) + 0.000001 
        return -(sum / norm1 / norm2)
    else:
        # ДЛЯ НЕ ПОЛНОСВЯЗНЫХ СЛОЕВ НУЖЕН ДРУГОЙ shape
        flat_output = tf.reshape(tensor=x_predict, shape=[x.shape.as_list()[0], -1])
        flat_input = tf.reshape(tensor=x_predict, shape=[x.shape.as_list()[0], -1])
        sum = tf.math.reduce_sum(tf.math.multiply(flat_output, flat_input), axis=-1)
        norm1 = tf.norm(flat_output, axis=-1) + 0.000001
        norm2 = tf.norm(flat_input, axis=-1) + 0.000001
        return -(tf.math.divide(tf.math.divide(sum, norm1), norm2))

## RSRAE model

In [29]:
import numpy as np
import tensorflow as tf
print("Tensorflow version = {}".format(tf.__version__)) # текущая версия tf

from tensorflow.keras import Model, optimizers, metrics
from tensorflow.keras.layers import Layer, Flatten, Dense, BatchNormalization

# from tensorflow.keras import activations, Sequential, Input
# from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape
# from sklearn.metrics import roc_auc_score, average_precision_score

from sklearn.metrics import roc_auc_score, average_precision_score

# Задаем random_seed для tensorflow и numpy
random_seed = 123
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

# Sets the default float type
tf.keras.backend.set_floatx('float64')

# Set random seed
tf.random.set_seed(123)
np.random.seed(123)


class RSR(Layer):
    """
    Robust Subspace Recovery (RSR) layer.
    Робастный слой, восстанавливающий подпространство. Задача данного слоя - отобразить
    закодированные энкодером данные в подпростраство так, чтобы после их обратного
    отображения декодером дивергенция между экземпляром исходных данных и его образом,
    полученным от автоэнкодера была незначительной для нормального экземпляра и была
    большой для аномального экземпляра. 

    # Example
    ```
        z_rsr, A = RSR(intrinsic_size=10)(z)
    ```
    # Arguments
        intrinsic_size: размерность z_rsr.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)` after encoding.
    # Output shape
        2D tensor with shape: `(n_samples, intrinsic_size)`.
    """

    def __init__(self, intrinsic_size: int, name="RSR_layer", **kwargs):
        super(RSR, self).__init__(name=name, **kwargs)
        # Если присваивать экземпляр слоя, как атрибут другого слоя, то хорошей
        # практикой делать создавать такие подслои в __init__ (поскольку подслои обычно
        # имеют метод build, они будут собраны, когда будет собран внешний слой). 
        self.flatten = Flatten()
        self.intrinsic_size = intrinsic_size
        
    def build(self, input_shape):
        """Определяет веса слоя, а именно задает матрицу A."""
        self.A = self.add_weight(name="A",
                                 shape=[int(input_shape[-1]), self.intrinsic_size],
                                 initializer='random_normal',
                                 trainable=True,)
        
    def call(self, z):
        """
        Логика слоя. Умножение выхода энкодера - вектора z на матрицу A.
        Возвращает отображенный z_rsr и матрицу A, которая потребуется далее.
        """
        z = self.flatten(z)
        z_rsr = tf.linalg.matmul(z, self.A) 
        return z_rsr

    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'intrinsic_size': self.intrinsic_size})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class L2Normalization(Layer):
    """Слой для l_2 нормализации, который будет применяться к выходу RSR layer."""
    
    def __init__(self, name="L2Normalization", **kwargs):
        super(L2Normalization, self).__init__(name=name, **kwargs)

    def call(self, z_rsr):
        """
        Выполняет l_2 нормализацию векторов, полученных после применения RSR layer
        вдоль оси, соответсвующей числу признаков. То есть производится нормализация
        каждого экземпляра выборки, в результате которой признаки экземпляров будут
        находиться в отрезке [-1; 1].
        """
        z_tilde = tf.math.l2_normalize(z_rsr, axis=-1)
        return z_tilde

    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Encoder(Layer):
    """
    Класс для encoder модели RSRAE. Отображает исходные данные input_data в вектор z,
    кодирующий исходные данные.
    """

    def __init__(self,
                 hidden_layer_dimensions,
                 activation,
                 flag_bn=True, 
                 name="Encoder",
                 **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.activation = activation
        self.flag_bn = flag_bn
        self.dense0 = Dense(hidden_layer_dimensions[0], activation=activation,
                            name='encoder_0')
        self.dense1 = Dense(hidden_layer_dimensions[1], activation=activation,
                            name='encoder_1')
        self.dense2 = Dense(hidden_layer_dimensions[2], activation=activation,
                            name='encoder_2')
        if flag_bn:
            self.batch_normalization0 = BatchNormalization(name="encoder_bn_layer_0")
            self.batch_normalization1 = BatchNormalization(name="encoder_bn_layer_1")
            self.batch_normalization2 = BatchNormalization(name="encoder_bn_layer_2")

    def call(self, inputs):
        """Отображние исходных данных x -> в закодированный вектор z."""
        x = inputs
        x = self.dense0(x)
        if self.flag_bn:
            x = self.batch_normalization0(x)
        x = self.dense1(x)
        if self.flag_bn:
            x = self.batch_normalization1(x)
        x = self.dense2(x)
        if self.flag_bn:
            x = self.batch_normalization2(x)
        z = x
        return z
    
    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'hidden_layer_dimensions': self.hidden_layer_dimensions})
        config.update({'activation': self.activation})
        config.update({'flag_bn': self.flag_bn})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Decoder(Layer):
    """
    Класс для decoder модели RSRAE. Отображает вектор z_rsr, полученный в результате
    кодирования исходных данных в вектор z, и последующим отображением вектора z при
    помощи RSR layer (x -> z -> z_rsr), обратно в пространство исходных данных 
    (z_rsr -> x_tilde).
    """

    def __init__(self,
                 inputs_dim,
                 hidden_layer_dimensions,
                 activation,
                 flag_bn=True, 
                 name="Decoder",
                 **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.activation = activation
        self.flag_bn = flag_bn
        self.dense2 = Dense(hidden_layer_dimensions[2], activation=activation,
                            name='decoder_2')
        self.dense1 = Dense(hidden_layer_dimensions[1], activation=activation,
                            name='decoder_1')
        self.dense0 = Dense(hidden_layer_dimensions[0], activation=activation,
                            name='decoder_0')
        self.dense_output = Dense(inputs_dim, activation=activation,
                            name='decoder_output')
        if flag_bn:
            self.batch_normalization2 = BatchNormalization(name="decoder_bn_layer_2")
            self.batch_normalization1 = BatchNormalization(name="decoder_bn_layer_1")
            self.batch_normalization0 = BatchNormalization(name="decoder_bn_layer_0")
    def call(self, inputs):
        """
        Отображние z_rsr -> x_tilde, где x_tilde - вектор, лежащий в пространстве
        исходных даных.
        """
        z_rsr = inputs
        z_rsr = self.dense2(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization2(z_rsr)
        z_rsr = self.dense1(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization1(z_rsr)
        z_rsr = self.dense0(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization0(z_rsr)
        x_tilde = self.dense_output(z_rsr)
        return x_tilde
    
    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'hidden_layer_dimensions': self.hidden_layer_dimensions})
        config.update({'activation': self.activation})
        config.update({'flag_bn': self.flag_bn})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class RSRAE(Model):
    """
    Нейросетевая модель-автоэнкодер для обнаружения аномалий с робастным слоем,
    восстанавливающим подпространство (RSR layer между encoder и decoder).
    Комбинируем encoder + RSR layer + decoder в end-to-end модель.
    """

    def __init__(self,
                 inputs_dim, # размерность вектора признаков
                 hidden_layer_dimensions,
                 intrinsic_size, # разерность z_rsr после RSR layer
                 activation,
                 flag_bn=True,
                 flag_normalize=True,
                 learning_rate=1e-3,
                 ae_loss_norm_type='MSE',
                 rsr_loss_norm_type='MSE',
                 name='RSRAE',
                 **kwargs):
        super(RSRAE, self).__init__(name=name, **kwargs)
        self.inputs_dim = inputs_dim
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.intrinsic_size = intrinsic_size
        self.activation = activation
        self.flag_bn = flag_bn
        self.flag_normalize = flag_normalize
        self.learning_rate = learning_rate
        self.ae_loss_norm_type = ae_loss_norm_type
        self.rsr_loss_norm_type = rsr_loss_norm_type
        # Для вычисления среднего loss по loss всех батчей в эпохе
        self.loss_tracker = metrics.Mean(name="loss")
        self.auc_tracker = metrics.Mean(name="auc")
        self.ap_tracker = metrics.Mean(name="ap")

        # Создание экземпляров оптимизаторов
        self.optimizer_ae = optimizers.Adam(learning_rate=learning_rate)
        self.optimizer_rsr1 = optimizers.Adam(learning_rate=10 * learning_rate)
        self.optimizer_rsr2 = optimizers.Adam(learning_rate=10 * learning_rate)

        # Слои
        self.encoder = Encoder(hidden_layer_dimensions=hidden_layer_dimensions,
                               activation=activation,
                               flag_bn=flag_bn)
        self.rsr = RSR(intrinsic_size=intrinsic_size)
        if flag_normalize:
            self.l2normalization = L2Normalization()
        self.decoder = Decoder(inputs_dim=inputs_dim,
                               hidden_layer_dimensions=hidden_layer_dimensions,
                               activation=activation,
                               flag_bn=flag_bn)
        
    def call(self, inputs):
        z = self.encoder(inputs)
        z_rsr = self.rsr(z)
        if self.flag_normalize:
            z_rsr = self.l2normalization(z_rsr)
        x_tilde = self.decoder(z_rsr)
        return z, z_rsr, x_tilde

    def ae_loss(self, x, x_tilde):
        """Функция потерь реконструкции автоэнкодера - L_AE."""

        x = tf.reshape(x, (tf.shape(x)[0], -1))
        x_tilde = tf.reshape(x_tilde, (tf.shape(x_tilde)[0], -1))

        # axis=1 для tf.norm => вычисление вдоль оси признаков
        # tf.math.reduce_mean без параметров - mean от элементов матрицы
        if self.ae_loss_norm_type in ['MSE', 'mse', 'Frob', 'F']:
            return tf.math.reduce_mean(tf.math.square(tf.norm(x-x_tilde, 
                                                              ord=2, axis=1)))
        elif self.ae_loss_norm_type in ['L1', 'l1']:
            return tf.math.reduce_mean(tf.norm(x-x_tilde, ord=1, axis=1))
        elif self.ae_loss_norm_type in ['LAD', 'lad', 'L21', 'l21', 'L2', 'l2']:
            return tf.math.reduce_mean(tf.norm(x-x_tilde, ord=2, axis=1))
        else:
            raise Exception("Norm type error!")
    
    def rsr1_loss(self, z, z_rsr):
        """Функция потери для RSR layer - L_RSR1."""
        z_rsr = tf.matmul(z_rsr, tf.transpose(self.rsr.A))

        if self.rsr_loss_norm_type in ['MSE', 'mse', 'Frob', 'F']:
            return tf.math.reduce_mean(tf.math.square(tf.norm(z-z_rsr, ord=2, 
                                                            axis=1)))
        elif self.rsr_loss_norm_type in ['L1', 'l1']:
            return tf.math.reduce_mean(tf.norm(z-z_rsr, ord=1, axis=1))
        elif self.rsr_loss_norm_type in ['LAD', 'lad', 'L21', 'l21', 'L2', 'l2']:
            return tf.math.reduce_mean(tf.norm(z-z_rsr, ord=2, axis=1))
        else:
            raise Exception("Norm type error!")
    
    def rsr2_loss(self):
        """Функция потери для RSR layer - L_RSR2."""
        A = self.rsr.A
        A_T = tf.transpose(A)
        I = tf.eye(self.intrinsic_size, dtype=tf.float64)
        return tf.math.reduce_mean(tf.math.square(tf.linalg.matmul(A_T, A) - I))
        
    def gradients(model, inputs, targets):
        with tf.GradientTape() as tape:
            loss_value = loss_fn(model, inputs, targets)
        return tape.gradient(loss_value, model.trainable_variables)
    
    @tf.function()
    def train_step(self, data):
        """
        Override the method. Будет вызываться при 'model.fit()'.
        Один шаг обучения, на котором вычисляются функции потерь для автоэнкодера и
        RSR layer, и в соотвествии с ними обновляются значения обучаемых переменных - 
        весов нейросети и матрицы A соответсвенно. Будет вызываться от одного батча.
        Заметим, что в этом методе мы используем пользовательские оптимизаторы и функции
        потерь, поэтому перед тренировкой метод compile вызывать не придется.
        """


        x, y = data

        # tf.GradientTape() - записывает операции для автоматического дифференцирования

        # По умолчанию persistent=False и удерживаемые GradientTape, высвобождаются,
        # как только вызывается метод GradientTape.gradient(). Чтобы вычислить несколько
        # градиентов за одно вычисление, требуется задать persistent=true. Это позволяет
        # многократно вызывать метод gradient(), тогда требуется самостоятельно
        # освободить ресурсы с помощью 'del tape'.

        # watch_accessed_variables=True => автоматическое отслеживание всех обучаемых
        # переменные, к которым осуществляется доступ. Так градиенты могут быть
        # запрошены c любого вычисленного результата в tape.
        with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tape:
            # Здесь требуется запустить прямой проход нейросети. Операции применяемые
            # при проходе к входных данным будут записаны на GradientTape. 
            z, z_rsr, x_tilde = self.call(x) # прямой проход RSRAE
            z = tf.keras.layers.Flatten()(z) # вроде для текстовых данных необязательно
            # Вычисляем значения функций потерь для этого прохода
            loss_ae = self.ae_loss(x, x_tilde)
            loss_rsr1 = self.rsr1_loss(z, z_rsr)
            loss_rsr2 = self.rsr2_loss()
  
        # Метод gradient вычисляет градиенты обучаемых параметров(весов) для минимизации
        # функции потерь, используя операции, записанные в контексте этого tape.
        gradients_ae = tape.gradient(loss_ae, self.trainable_weights)
        gradients_rsr1 = tape.gradient(loss_rsr1, self.rsr.A)
        gradients_rsr2 = tape.gradient(loss_rsr2, self.rsr.A)

        # Обновим значения обучаемых переменных - градиентный шаг чтобы min loss.
        self.optimizer_ae.apply_gradients(grads_and_vars=
                                          zip(gradients_ae, self.trainable_weights))
        self.optimizer_rsr1.apply_gradients(grads_and_vars=
                                            zip([gradients_rsr1], [self.rsr.A]))
        self.optimizer_rsr2.apply_gradients(grads_and_vars=
                                            zip([gradients_rsr2], [self.rsr.A]))
        
        self.loss_tracker.update_state(loss_ae) # обновляем средний loss по батчам

        # Обновляем метрики
        if len(tf.unique(y)[0]) == 2:
            # иначе roc_auc_score бросит ValueError и обучение приостановится
            auc = self.auc_metric(y, cosine_similarity(x_tilde, x))
            self.auc_tracker.update_state(auc)

        ap = self.ap_metric(y, cosine_similarity(x_tilde, x))
        self.ap_tracker.update_state(ap)

        del tape # persistent=True => требуется самостоятельно освободить ресурсы
        return {"loss": self.loss_tracker.result(),
                "auc": self.auc_tracker.result(),
                "ap": self.ap_tracker.result(),}

    @property
    def metrics(self):
        """
        В пару к train_step. Сбрасывает метрики (`reset_states()`) в начале каждой
        эпохи обучения с помощью 'fit()'. Без этого свойства 'result()' будет 
        возвращать среднее значение с начала обучения.
        """
        return [self.loss_tracker, self.auc_tracker, self.ap_tracker]

    def auc_metric(self, y_true, y_pred):
        return tf.py_function(roc_auc_score, (y_true, y_pred), tf.float64)

    def ap_metric(self, y_true, y_pred):
        return tf.py_function(average_precision_score, (y_true, y_pred), tf.float64)

Tensorflow version = 2.3.0


In [30]:
model_rsrae = RSRAE(inputs_dim=x.shape[-1],
                    hidden_layer_dimensions=[128, 256, 512],
                    intrinsic_size=50,
                    activation='relu',
                    learning_rate=25e-5,
                    ae_loss_norm_type='MSE',
                 rsr_loss_norm_type='MSE',)
model_rsrae.compile(run_eagerly=True)
model_rsrae.fit(x, y,
                batch_size=128,
                epochs=500)

Epoch 1/500
9/9 [==============================] - 1s 107ms/step - loss: 1.0096 - auc: 0.5416 - ap: 0.1787
Epoch 2/500
9/9 [==============================] - 0s 27ms/step - loss: 0.9891 - auc: 0.5220 - ap: 0.1565
Epoch 3/500
9/9 [==============================] - 0s 26ms/step - loss: 0.9852 - auc: 0.5605 - ap: 0.1889
Epoch 4/500
9/9 [==============================] - 0s 26ms/step - loss: 0.9836 - auc: 0.5958 - ap: 0.2151
Epoch 5/500
9/9 [==============================] - 0s 26ms/step - loss: 0.9824 - auc: 0.6268 - ap: 0.2585
Epoch 6/500
9/9 [==============================] - 0s 26ms/step - loss: 0.9812 - auc: 0.6424 - ap: 0.2720
Epoch 7/500
9/9 [==============================] - 0s 25ms/step - loss: 0.9797 - auc: 0.6695 - ap: 0.2970
Epoch 8/500
9/9 [==============================] - 0s 25ms/step - loss: 0.9775 - auc: 0.6708 - ap: 0.2859
Epoch 9/500
9/9 [==============================] - 0s 26ms/step - loss: 0.9744 - auc: 0.6819 - ap: 0.2961
Epoch 10/500
9/9 [===========================

In [31]:
x_predict = model_rsrae.predict(x)[2]

auc = roc_auc_score(y, cosine_similarity(x_predict, x))
ap = average_precision_score(y, cosine_similarity(x_predict, x))
                                    
print("auc = ", auc)
print("ap = ", ap)

auc =  0.6618016644693472
ap =  0.23899051205435262
